In [1]:
from datetime import datetime
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sn

import bee_utils as bee

pd.set_option("display.precision", 2)

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 10,
    'text.usetex': True,
    'pgf.rcfonts': False,
})


In [2]:
PN_BASE_DIR = Path("../Pointnet_Pointnet2_pytorch/log/classification/")
# PRED_FILE = PN_BASE_DIR / "msg_cls5C_e40_bs8_pts4096_split7030_ds5fps/logs/pred_per_sample_7030.csv"
PRED_FILE = PN_BASE_DIR / "msg_cls6_e40_bs8_pts4096_split7030_ds5fps/logs/pred_per_sample_7030.csv"

exp_name = PRED_FILE.parent.parent.name

df = pd.read_csv(PRED_FILE, header="infer")

display(df)

,sample_path,target_name,pred_name,insect,bee,butterfly,dragonfly,wasp,bumblebee
0,bee\hn-bee-1_3_1.csv,bee,bee,-3.05,-0.43,-7.47,-4.33,-7.50,-1.26
1,bee\hn-bee-1_3_2.csv,bee,bumblebee,-2.96,-1.16,-3.73,-2.20,-4.99,-0.77
2,bee\hn-bee-1_3_3.csv,bee,bumblebee,-3.76,-3.43,-4.47,-2.32,-4.84,-0.19
3,bee\hn-bee-1_3_4.csv,bee,insect,-0.10,-3.75,-6.11,-4.76,-2.91,-5.70
4,bee\hn-bee-1_3_5.csv,bee,insect,-0.05,-4.00,-6.31,-8.91,-3.69,-6.30
...,...,...,...,...,...,...,...,...,...
567,wasp\hn-was-5_51_1.csv,wasp,wasp,-2.90,-6.16,-8.13,-2.24,-0.18,-9.54
568,wasp\hn-was-5_51_2.csv,wasp,insect,-0.76,-3.52,-5.04,-1.62,-1.28,-6.79
569,wasp\hn-was-5_51_4.csv,wasp,bee,-1.80,-0.94,-7.45,-6.34,-1.05,-3.08
570,wasp\hn-was-5_53_1.csv,wasp,wasp,-3.73,-5.64,-8.35,-4.65,-0.04,-9.40


In [3]:
def abbr_class_name(cl, mapp=None):
    if mapp is not None and cl in mapp:
        return mapp[cl]
    if cl in bee.CLASSES and len(cl)>3:
        return cl[:3]+"."
    else:
        return cl
    
mapp1 = {
    "bee":"bee","bumblebee":"bumbleb.","wasp":"wasp","dragonfly":"dragonfly","butterfly":"butterfly","insect":"insect"
}

In [4]:


all_classes_ordered = ["bee","bumblebee","wasp","dragonfly","butterfly","insect"]
classes_ordered = [cl for cl in all_classes_ordered if cl in df.columns]

df_reordered = df[classes_ordered]

# TODO change "bublebee" to "bumbleb." otherwise the name is too long for the plot.
# TODO smaller font

# Build confusion matrix
cf_matrix = confusion_matrix(df["target_name"], df["pred_name"], labels=classes_ordered)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [abbr_class_name(i,mapp1) for i in classes_ordered],
                     columns = [abbr_class_name(i,mapp1) for i in classes_ordered])
plt.figure(figsize = (5,3.2))
ax = sn.heatmap(df_cm, annot=True, cmap="Blues")
ax.set_ylabel("Ground-Truth")
ax.set_xlabel("Klassifikation")

ax.get_figure().show()


C:\Users\Marc\AppData\Local\Temp\ipykernel_6704\1370907060.py:18: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  ax.get_figure().show()


In [5]:
# save for latex


fig = ax.get_figure()
fig.tight_layout()
# fig.set_size_inches(6.3,5)
datetime_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = Path(f'output/from_notebooks/cm/{datetime_str}_{exp_name}')
output_dir.mkdir(exist_ok=True, parents=True)
fig.savefig(str(output_dir / f"cm_{exp_name}.png"))
fig.savefig(str(output_dir / f"cm_{exp_name}.pgf"))


################### WICHTIG! #####################
# Auch generiertes ...-img0.png muss kopiert werden!
# In PGF Datei muss Pfad von ...-img0.png angepasst werden:
# Davor muss das Verzeichnispfad im Latex-res-Dir angegeben werden!
# ZB: {res/pgf/cm_msg_cls5C_e40_bs8_pts4096_split7030_ds5fps-img0.png}
##################################################